In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import csv, datetime, os
import matplotlib.dates as mdates
import pandas as pd
from sklearn.metrics import r2_score
import pandas as pd
from tqdm import tqdm


def MakeDataFromText(filepath:str):
    """
    Helper function to parse text files in IFM proprietary format.
    """
    with open(filepath, 'r') as file:
        reader = csv.reader(file)

        db = dict()                          # Dictionary that will hold k,v pairs of 
        recent_EPC = ''                      # The most recent EPC that was read, and data is being assigned to
        Time_Or_Data = True                  # Time is true, Data is false
        reached_24 = False

        for l in reader:                     # Iterate over entire text file
            line = l[0]                      # Each line only has one token, so we select it
            
            if len(line)==24:                # If it is an EPC
                db[line] = [ [],[] ]         # Create new k,v pair in database
                recent_EPC = line            # Current EPC to add to is selected
            
            elif line == '[': data = []      # If start bracket, beginning to collect an array of data

            elif line == ']':                # End bracket means end of data array
                if len(data)!=0:
                    if Time_Or_Data:
                        db[recent_EPC][0] = data
                        Time_Or_Data = False
                    else:
                        db[recent_EPC][1] = data
                        Time_Or_Data = True
                data = [ ]

            elif line.count(":")==2:
                h,m,s = line.split(':')      # Parse timestamp into hours, minutes, seconds

                if int(h)==23:
                    reached_24 = True        # If end of day reached, must move to Day 2

                # if reached_24 and int(h) < 12:
                #     day=2
                # else:
                #     day=1

                day = 1

                data.append(datetime.datetime(1970, 2, day, int(h), int(m), int(s)))
            
            elif line ==' ': pass          # If blank space, then we skip to parse next line

            else:                          # If it is a data point
                try:
                    data.append(float(line))
                except:
                    pass                   # Error catching case just in case, so annoying errors don't crash program
    return db


def MakeDict(data:list):
    """
    """
    t1234 = dict()
    for d in data:
        for k,v in d.items():
            if k in t1234: # if tag is already in dict
                for i in range(len(v[0])):
                    if v[0][i] in t1234[k]:
                        t1234[k][v[0][i]].append(v[1][i])
                    else:
                        t1234[k][v[0][i]] = [v[1][i]]
            else:             # if tag is not in dict
                t1234[k] = dict()
                for i in range(len(v[0])):
                    t1234[k][v[0][i]] = [v[1][i]]
    for k,v in t1234.items():
        for k2,v2 in v.items():
            t1234[k][k2] = sum(v2)/len(v2)
    return t1234


def MakeShirt(epcs:list)->dict:
    """
    Create a dictionary mapping EPCs to their respective shirt body parts.
    """
    parts = [ "Back Neck", "Back", "Chest", "Left Ab", "Right Ab", "Right Up Arm", "Right Low Arm", "Left Up Arm", "Left Low Arm" ]
    output_dict = dict()
    for i in range(len(epcs)):
        # output_dict[parts[i]] = epcs[i]
        output_dict[epcs[i]] = parts[i]
    return output_dict


def MakeGlove(epcs:list)->dict:
    """
    Create a dictionary mapping EPCs to their respective glove fingers.
    """
    parts = [ "Forefinger", "Pinkie" ]
    output_dict = dict()
    for i in range(len(epcs)):
        # output_dict[parts[i]] = epcs[i]
        output_dict[epcs[i]] = parts[i]
    return output_dict


def MakeHeadGear(epcs:list)->dict:
    """
    Create a dictionary mapping EPCs to their respective headgear body parts.
    """
    parts = 8*[ 'Beanie' ] + 4*[ 'Balaclava']
    output_dict = dict()
    for i in range(len(epcs)):
        output_dict[epcs[i]] = parts[i]
    return output_dict

In [ ]:
march1  = MakeDict([MakeDataFromText('Relevant data/1st March/tags.txt')])
feb23   = MakeDict([MakeDataFromText('Relevant data/23th Feb/tags.txt')])
feb24_1 = MakeDict([MakeDataFromText('Relevant data/24th Feb/tags.txt')])
feb24_2 = MakeDict([MakeDataFromText('Relevant data/24th Feb part 2/tags.txt')])

In [ ]:
# Hefner, Glove   - 1, 2, 5
# Gelinas, Glove  - 7, 8, 9
# Person 3, Glove - 6, 11, 12
glove1_L  = MakeGlove( ["7251", "577F"] ); glove1_R  = MakeGlove( ["8DC7", "4DA7"] );
glove2_L  = MakeGlove( ["5A93", "1342"] ); glove2_R  = MakeGlove( ["69AF", "A074"] );
glove5_L  = MakeGlove( ["3FCE", "39A0"] ); glove5_R  = MakeGlove( ["5E18", "6423"] );
glove6_L  = MakeGlove( ["6B69", "3BB8"] ); glove6_R  = MakeGlove( ["1235", "422D"] );
glove7_L  = MakeGlove( ["087D", "3D35"] ); glove7_R  = MakeGlove( ["3B9A", "8582"] );
glove8_L  = MakeGlove( ["59A9", "9742"] ); glove8_R  = MakeGlove( ["5198", "49D5"] );
glove9_L  = MakeGlove( ["0A70", "2CAB"] ); glove9_R  = MakeGlove( ["2D5B", "4548"] );
glove11_L = MakeGlove( ["0AA9", "6904"] ); glove11_R = MakeGlove( ["7A40", "55D5"] );
glove12_L = MakeGlove( ["3B21", "1E46"] ); glove12_R = MakeGlove( ["794B", "3ACE"] );


# Hefner, L   - 1, 4, 8, 9
# Gelinas, M  - 2, 3, 11
# Person 3, L - 5, 6, 10
shirt1  = MakeShirt( ["7F57", "6082", "51BE", "0551", "5D88", "1EB8", "5CA6", "89BA", "5286"] );
shirt2  = MakeShirt( ["3259", "846D", "0469", "8C94", "53C5", "34D5", "36C1", "8534", "0587"] );
shirt3  = MakeShirt( ["82BD", "A892", "7A48", "4D1E", "849B", "0D83", "5C9A", "78AE", "877F"] );
shirt4  = MakeShirt( ["26C0", "3CA6", "3D5B", "1D8D", "7C8A", "4768", "843F", "2846", "4257"] );
shirt5  = MakeShirt( ["4594", "1073", "3415", "56AE", "6809", "97A8", "9B3D", "917C", "6627"] );
shirt6  = MakeShirt( ["89BE", "522F", "3D80", "3F51", "597F", "8599", "80DC", "026C", "B574"] );
shirt8  = MakeShirt( ["30CB", "3592", "3B18", "75D4", "54D3", "5F3A", "8A4C", "73A1", "4CA2"] );
shirt9  = MakeShirt( ["859F", "A75A", "AF4F", "4946", "5AAA", "5FAF", "5C89", "A958", "B66D"] );
shirt10 = MakeShirt( ["1772", "0385", "5487", "1A30", "482E", "4FDF", "5A34", "73CD", "92A1"] );
shirt11 = MakeShirt( ["9D7A", "8913", "A587", "B894", "5988", "1C82", "0088", "AC59", "382A"] );

hefnerhead  = MakeHeadGear( ["639A", "7971", "B878", "7819", "6D60", "19BF", "6221", "5EB7", "5410", "47D4", "928A", "59CF"] );
gelinashead = MakeHeadGear( ["B365", "348A", "7166", "398F", "6818", "4D9D", "9B90", "9D39", "8A77", "A45B", "96C0", "9DA6"] );
person3head = MakeHeadGear( ["77B2", "3E64", "4C2D", "3848", "2CA4", "5FDB", "564E", "62D2", "2F77", "A382", "304D", "7E48"] );

# List<string> Hefner_Beanie  = new List<string>{ "639A", "7971", "B878", "7819", "6D60", "19BF", "6221", "5EB7" };
# List<string> Gelinas_Beanie = new List<string>{ "B365", "348A", "7166", "398F", "6818", "4D9D", "9B90", "9D39" };
# List<string> Person3_Beanie = new List<string>{ "77B2", "3E64", "4C2D", "3848", "2CA4", "5FDB", "564E", "62D2" };
# List<string> Hefner_Bala  = new List<string>{ "5410", "47D4", "928A", "59CF" };
# List<string> Gelinas_Bala = new List<string>{ "8A77", "A45B", "96C0", "9DA6" };
# List<string> Person3_Bala = new List<string>{ "2F77", "A382", "304D", "7E48" };

Hefner_Shirts = [ shirt1, shirt4, shirt8, shirt9 ]
Hefner_Names  = [ "Shirt 1", "Shirt 4", "Shirt 8", "Shirt 9" ]
Gelinas_Shirts = [ shirt2, shirt3, shirt11 ]
Gelinas_Names  = [ "Shirt 2", "Shirt 3", "Shirt 11" ]
Person3_Shirts = [ shirt5, shirt6, shirt10 ]
Person3_Names  = [ "Shirt 5", "Shirt 6", "Shirt 10" ]

Hefner_Gloves = [ glove1_L, glove1_R, glove5_L, glove5_R, glove2_L, glove2_R ]
Hefner_Glove_Names = [ "Glove 1 Left", "Glove 1 Right", "Glove 5 Left", "Glove 5 Right", "Glove 2 Left", "Glove 2 Right" ]

In [ ]:
def ColorPicker(bodypart:str)->str:

    if bodypart=="Beanie":      # Dark Red for Beanie
        return "#C40000"  

    if bodypart=="Balaclava":   # Dark Orange for Balaclava
        return "#E39700"
    
    if bodypart=="Pinkie":      # Hot Pink for Pinkine
        return "#F101D4"
    
    if bodypart=="Forefinger":  # Yellow for Forefinger
        return "#bfbc04"
    
    colordict = {
        "Back Neck": "#20ed00", # Greens for back and chest
        "Back": "#11910a",
        "Chest": "#006118",

        "Left Ab": "#0269ad",   # Blues for abs, this one more green
        "Right Ab": "#020ee3",  # more royal blue

        "Right Up Arm": "#0e00e3",
        "Right Low Arm": "#6a00e3",
        "Left Up Arm": "#e300e3",
        "Left Low Arm": "#e3006a"
    }

    return colordict[bodypart]

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 7), sharey=True)
axs = [ax1, ax2]

# DAYS = [ feb24_1, feb24_2 ]
DAYS = [ feb24_2, feb24_1 ]
# DATA = [ gelinashead, shirt2, shirt3, shirt11 ]
DATA = [ hefnerhead, shirt1, shirt4, shirt8, shirt9 ]

for D in range(2):
    for k,v in DAYS[D].items():
        for i in range(len(DATA)):
            if k[-4:] in DATA[i].keys():
                st = sorted(DAYS[D][k].items())
                dt, tt = [j[1] for j in st], [j[0] for j in st]

                axs[D].plot(tt, dt, label=DATA[i][k[-4:]] + ", " + k[-4:], marker=".", color=ColorPicker(DATA[i][k[-4:]]) )
                axs[D].set_title("Hefner Garments, Feb 24 Part " + str(D+1), fontsize=20)

ax1.plot([ 
    datetime.datetime(1970, 2, 1, 14, 17, 10), datetime.datetime(1970, 2, 1, 14, 18, 20) ], 
    [18.0, 18.0], color="red", label="Ambient Temp", linestyle="--"
)

ax2.plot([ 
    datetime.datetime(1970, 2, 1, 15, 9, 50), datetime.datetime(1970, 2, 1, 15, 11, 10) ], 
    [-20.0, -20.0], color="red", label="Ambient Temp", linestyle="--"
)

for a__ in [ax1, ax2]:
    a__.xaxis.set_major_formatter( mdates.DateFormatter('%H:%M:%S') )
    a__.legend()
    a__.set_xlabel("Time")
    a__.set_ylabel("Temperature (C)")
    # a__.set_xticklabels(rotation=45)

fig.subplots_adjust(wspace=0.2)

In [ ]:
# DAYS = [ march1, feb23, feb24_1, feb24_2 ]
DAYS = [feb23]

SHIRT = Gelinas_Shirts

DATA = {
    "Back Neck" : [], 
    "Back" : [], 
    "Chest" : [], 
    "Left Ab" : [], 
    "Right Ab" : [], 
    "Right Up Arm" : [], 
    "Right Low Arm" : [], 
    "Left Up Arm" : [], 
    "Left Low Arm" : []
}

for DAY in DAYS:
    for k,v in DAY.items():
        for i in range(len(SHIRT)):
            if k[-4:] in SHIRT[i].keys():
                st = sorted(DAY[k].items())
                dt, tt = [j[1] for j in st], [j[0] for j in st]
                # print(SHIRT[i][k[-4:]])
                for d_ in dt:
                    DATA[SHIRT[i][k[-4:]]].append(d_)
                
for k,v in DATA.items():
    print(
        round(np.mean(v),2),  round(np.std(v),2), len(v), k
    )

In [ ]:
fig, (ax1, ax2, ax3, ax4) = plt.subplots(4, 1, figsize=(18, 14), sharex=True)
ax1.xaxis.set_major_formatter( mdates.DateFormatter('%H:%M:%S') )
ax2.xaxis.set_major_formatter( mdates.DateFormatter('%H:%M:%S') )
ax3.xaxis.set_major_formatter( mdates.DateFormatter('%H:%M:%S') )
ax4.xaxis.set_major_formatter( mdates.DateFormatter('%H:%M:%S') )

axs = [ax1, ax2, ax3, ax4]

DAY = feb23

for k,v in DAY.items():
    for i in range(len(Hefner_Shirts)):
        if k[-4:] in Hefner_Shirts[i].keys():
            st = sorted(DAY[k].items())
            dt, tt = [j[1] for j in st], [j[0] for j in st]

            axs[i].set_title(Hefner_Names[i])
            axs[i].plot(tt, dt, label=Hefner_Names[i] + ", " + Hefner_Shirts[i][k[-4:]], marker=".")
            # axs[i].set_title(Hefner_Names[i] + ", February 23rd")

fig.subplots_adjust(hspace=0.25)
for a__ in axs:
    a__.legend()
    a__.set_xlabel("Time")
    a__.set_ylabel("Temperature (C)")

In [ ]:
fig, ax = plt.subplots(1,1 , figsize=(9, 7), sharex=True)

# mean = [ 20.59, 21.23, 20.61, 20.15, 22.67, 20.99, 22.09, 21.89, 21.10 ]
# stds = [ 1.51, 1.87, 0.61, 0.35, 1.88, 2.67, 1.70, 2.37, 3.15 ]
mean = [16.87, 16.92, 22.35, 18.09, 15.97, 17.15, 18.18, 21.96, 21.22]
stds = [ 1.05, 0.32, 2.05, 3.82, 1.48, 1.63, 4.16, 5.01, 1.66 ]
color = 2*['#9709DD'] + 3*['#036E30'] + 4*["#E09F01"]

for i in range(len(mean)):
    ax.scatter (i, mean[i], marker='o',   color=color[i])
    ax.errorbar(i, mean[i], yerr=stds[i], color=color[i])

ax.plot([0,8], [21.2, 21.2], color='red', linestyle='--')

ax.set_xticklabels(
    [ "", "Back Neck", "Back", "Chest", "Left Ab", "Right Ab", "Right\nUp Arm", "Right\nLow Arm", "Left\nUp Arm", "Left\nLow Arm" ]
)